# Named Entity Recognition (NER)

I will attempt to do the Named Entity Recognition task. I will train two models and demonstrate their results below.

The models are:<br>
1- spaCy NER Gold Parse system [1]. spaCy is an industrial strength NLP library that is widely used by large corporation to handle similar NLP applications. E.g. it's being used by BBC, Microsoft, and many other big companies.

2- Flair embeddings [2]. Contextual String Embeddings for Sequence Labeling is currently the state-of-the-art [3] system in Named Enitiy Recognition task, and the only system outperforming Google's BERT [4] model. More information on Flair can be found on their paper [5].

I have selected these two models to demonstrate that I am capable of providing a quick & relaiable solution when needed (spaCy). Also, when time/resource allows, I am capable of providing a siginfacntly better solution that is considered the state-of-the-art in the field of NLP (Flair embeddings).

spaCy needs a couple of hours to train a decent model on a potato laptop, while Flair embeddings (with CharLMEmbeddings) can take a couple of days on a proper workstation with GPUs. 

Requirements to run this code:
- python 3.6
- spacy '2.0.16'
- flair

[1] https://spacy.io/<br>
[2] https://github.com/zalandoresearch/flair<br>
[3] https://github.com/zalandoresearch/flair#comparison-with-state-of-the-art<br>
[4] https://ai.googleblog.com/2018/11/open-sourcing-bert-state-of-art-pre.html<br>
[5] https://drive.google.com/file/d/17yVpFA7MmXaQFTe-HDpZuqw9fJlmzg56/view)

## Initialization

In [1]:
import spacy
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy import displacy
import json
import random
from flair.data import Sentence
from flair.models import SequenceTagger
from flair.trainers import SequenceTaggerTrainer
from flair.data import TaggedCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, WordEmbeddings, StackedEmbeddings, CharLMEmbeddings, CharacterEmbeddings
from typing import List

## Training, Validation, Testing

In here we divide the provided training data into 3 parts; training (80%), validation (10%) and testing (10%).

In [2]:
# reading the raw data
raw_data_PATH = '../NER/Dataset/'
File_ = open(raw_data_PATH+'ner_dataset.txt')

DATA = []
sentence = []

for line in File_:
    try:
        if line == '\n':
            DATA.append(sentence)
            sentence = []
        else:
            sentence.append(line)
    except:
        print('you have a bad line..',line)

random.shuffle(DATA)    # shuffling the data is always good to preven overfitting

# dividing the data into trainig (80%), validation (10%) and testing (10%).
split_ = int(0.1 * len(DATA))
TRAIN_DATA, VAL_DATA, TEST_DATA = DATA[:8*split_], DATA[8*split_:9*split_], DATA[9*split_:]
print('  *Training has (',len(TRAIN_DATA),') instances.')
print('  *Validation has (',len(VAL_DATA),') instances.')
print('  *TEST has (',len(TEST_DATA),') instances.')

# print()
# storing the data
for name,data in zip(['training','validation','test'],[TRAIN_DATA, VAL_DATA, TEST_DATA]):
    F = open(raw_data_PATH+'ner_dataset_'+name+'.txt','w')
    F.write('\n'.join([''.join(x) for x in TRAIN_DATA]))
    F.close()


  *Training has ( 14760 ) instances.
  *Validation has ( 1845 ) instances.
  *TEST has ( 1849 ) instances.


## 1- spaCy NER

### Preparing the data for spacy

Inspecting the data, shuffeling it and preparing it to be fed into the Spacy model.

In [3]:
# theses functions create training data suitable for the Spacy tool
def _reformat_data(data):
    for counter, example_ in enumerate(data):
        index_ = 0
        annotations = {}
        sentence, ner_tag = example_
        for word, tag in zip(sentence, ner_tag):
            #-------------------------------------#
            # analysing the NER tag
            if '-' in tag:
                In, tag = tag.split('-')
                if tag not in annotations:
                    annotations[tag] = []
            else:
                In = tag
                
            #-------------------------------------#
            # creating the training data
            if In == 'B':
                annotations[tag].append([index_, index_+len(word)])
            elif In == 'I':
                annotations[tag][-1][1] = index_+len(word)
            elif In != 'O':
                print('=====!!!!!', In)
                
            index_ += len(word) + 1
        
        # fix the format
        ann = {'entities':[ (val[0],val[1],key) for key in annotations for val in annotations[key]]}
            
        ## update the training data to fit spacy format
        text = ' '.join(sentence)
        data[counter] = (text, ann)
    return data

def _create_training_data(raw_data):
    File_ = open(raw_data, 'r')
    TRAIN_DATA = []
    sentence = []
    ner_tag = []

    for line in File_:
        try:
            line = line.split('\n')[0]

            if line == '':
                TRAIN_DATA.append([sentence,ner_tag])
                sentence = []
                ner_tag = []
            else:
                word, POS1, CNK2, tag = line.split(' ')
                sentence.append(word)
                ner_tag.append(tag)
        except:
            print('you have a bad line..',line)
            
    File_.close()
    return _reformat_data(TRAIN_DATA)
            

In [4]:
# reading the raw data
raw_data_PATH = '../NER/Dataset/'

# prepare the training data into spacy format
TRAIN_DATA = _create_training_data(raw_data_PATH+'ner_dataset_training.txt')
VAL_DATA = _create_training_data(raw_data_PATH+'ner_dataset_validation.txt')
TEST_DATA = _create_training_data(raw_data_PATH+'ner_dataset_test.txt')

# Inspecting the data        
print('We have a total of',len(TRAIN_DATA),'training instances.')

# print class analysis
all_tags = [ent[2] for data in TRAIN_DATA for ent in data[1]['entities']]
classes = set(all_tags)
print('We have',len(classes),'classes in this dataset.',classes)

for c_ in classes:
    print('  -class(',c_,') has',all_tags.count(c_),'instances.')


We have a total of 14759 training instances.
We have 4 classes in this dataset. {'LOC', 'PER', 'MISC', 'ORG'}
  -class( LOC ) has 7246 instances.
  -class( PER ) has 6732 instances.
  -class( MISC ) has 3493 instances.
  -class( ORG ) has 6075 instances.


### Training the model

In [ ]:
# Load or create a blank English model
model = '../NER/Spacy/'
output_dir = '../NER/Spacy/'

if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
        

"""Load the model, set up the pipeline and train the entity recognizer."""
if model is not None:
    try:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    except:
        nlp = spacy.blank('en')  # create blank Language class
        print("Could not find the model, Created a blank 'en' model isntead")
else:
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")
    

Could not find the model, Created a blank 'en' model isntead


In [ ]:
def predict_on_texts(texts):
    colors = {}
    colors['ORG'] = 'orange'
    colors['PER'] = '#aa9cfc'
    colors['LOC'] = 'green'
    colors['MISC'] = 'yellow'
    options = {'ents': classes, 'colors': colors}

    for text in texts:
        doc = nlp(text)
        Entities = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        if len(Entities) > 0:
            displacy.render(doc, style='ent', jupyter=True, options=options)
        else:
            print('no entities detected: ',text)
        print('--------------------------')
        print()
    

In [ ]:
def predict_on_test_set(filepath):
    ext = filepath.split('.')[-1]
    if ext == 'json':
        VAL_DATA = json.load(open(filepath,'r'))
    elif ext == 'txt':
        VAL_DATA = _create_training_data(filepath) 
    else:
        VAL_DATA = []

    TP, FN, FP = 0, 0, 0 # True positives, False negatives, False Positives
    for text, ann in VAL_DATA:
        doc = nlp(text)
        GT = sorted(ann['entities'], key=lambda tup: tup[0])
        Entities = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        Ground_Truth = [(text[a[0]:a[1]], a[0], a[1], a[2]) for a in GT]
        
        TP += len([value for value in Entities if value in Ground_Truth])
        FP += len([value for value in Entities if value not in Ground_Truth])
        FN += len([value for value in Ground_Truth if value not in Entities])
    Pr, Re = TP/(TP+FP), TP/(TP+FN) ## computing Precision and Recall
    print('  -Validation: -precision=%.3f -recall=%.3f -f1 score=%.3f'  % (Pr, Re, 2*(Pr*Re)/(Pr+Re)))
        

In [ ]:
# parameters
n_iter = 20 # number of iteration
                
# create the built-in pipeline components and add them to the pipeline
# nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')
    
# add labels to model
for ent in classes:
    ner.add_label(ent)

In [ ]:
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    print('*started trianing..')
    for itn in range(n_iter):
        # shuffle the data (reduce the overfitting of the model)
        random.shuffle(TRAIN_DATA)
        losses = {}
        # batch up the examples using spaCy's minibatch
        batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,  # batch of texts
                annotations,  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        ## printing training and validation losses
        print('  -Trainnig loss', losses)
        predict_on_test_set(raw_data_PATH+'ner_dataset_validation.txt')
        
        # save model to output directory
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)



*started trianing..
  -Trainnig loss {'ner': 544.528265735968}
  -Validation: -precision=0.772 -recall=0.784 -f1 score=0.778
Saved model to ../NER/Spacy
  -Trainnig loss {'ner': 196.13384832894357}
  -Validation: -precision=0.887 -recall=0.903 -f1 score=0.895
Saved model to ../NER/Spacy
  -Trainnig loss {'ner': 134.60106617566413}
  -Validation: -precision=0.926 -recall=0.931 -f1 score=0.929
Saved model to ../NER/Spacy


#### Testing the model

In [ ]:
# To test the model on a txt file use:
# predict_on_test_set(raw_data_PATH+'ner_dataset.txt')

# To test the model with a sequence of sentences use:
predict_on_texts(['New York city','My name is Muhannad, and I live in the US. I work in Rolls Royce.'])


In [ ]:
# Plotting the NER tags.
# using display-spacy to show the ner values (showing a sample only).
random_examples = [int(random.random()*len(VAL_DATA)) for i in range(4)]
texts = [VAL_DATA[ex][0] for ex in random_examples]
predict_on_texts(texts)

## Flair model

Flair includes word embeddings to predict the named entites within the text.

In [ ]:
# define columns
columns = {0: 'text', 1: 'pos', 2: 'cnk', 3:'ner'}

# this is the folder in which train, test and dev files reside
data_folder = '../NER/Dataset/'

# training folder
flair_folder = '../NER/Flair/'
flair_path = Path(flair_folder)
if not flair_path.exists():
    flair_path.mkdir()
        
# retrieve corpus using column format, data folder and the names of the train, dev and test files
# 1. get the corpus
corpus: TaggedCorpus = NLPTaskDataFetcher.fetch_column_corpus(data_folder, columns,
                                                              train_file='ner_dataset_training.txt',
                                                              test_file='ner_dataset_test.txt',
                                                              dev_file='ner_dataset_validation.txt')
                
# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

# 4. initialize embeddings
embedding_types: List[TokenEmbeddings] = [

    # use this if you have a potato PC
    WordEmbeddings('glove'),

    # comment in this line to use character embeddings
    # CharacterEmbeddings(),

    # comment in these lines to use contextual string embeddings
#     CharLMEmbeddings('news-forward'),
#     CharLMEmbeddings('news-backward'),
]

embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)
    

# 5. initialize sequence tagger
tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)
    

# 6. initialize trainer
trainer: SequenceTaggerTrainer = SequenceTaggerTrainer(tagger, corpus)

# 7. start training
trainer.train(flair_folder,
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=20)

